In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
def Bollinger_Bands(df,multiplier=2,column='Close',window=20):
  df["MA"]=df[column].rolling(window=window).mean()
  df['STD'] = df['Close'].rolling(window=window).std()
  df['Upper'] = df['MA'] + (multiplier * df['STD'])
  df['Lower'] = df['MA'] - (multiplier * df['STD'])
  plt.figure(figsize=(12,6))
  plt.plot(data[column],label="Close_Price",color="blue")
  plt.plot(data['Lower'], label='Lower Band', color='green', linestyle='--')
  plt.plot(data['MA'], label='Moving Average', color='black')
  plt.fill_between(data.index, data['Lower'], data['Upper'], color='gray', alpha=0.2)
  plt.title('Bollinger Bands')
  plt.xlabel('Date')
  plt.ylabel('Price')
  plt.show()
  df["Signals"]=0
  df.loc[df[column]>df["Upper"],"Signals"]=-1
  df.loc[df[column]<df["Lower"],"Signals"]=1
  return df['Signals']

In [5]:
def MACD(df,fast=12,slow=36,signal=9,columns="Close"):
  df["EMA_Fast"]=df[column].ewm(span=fast,adjust=false).mean()
  df["EMA_Slow"]=df[column].ewm(span=slow,adjust=false).mean()
  df["MACD"]=df["EMA_Fast"]-df["EMS_Slow"]
  df["Signal_Line"]=df["MACD"].ewm(span=signal,adjust=False).mean()
  df['Histogram'] = df['MACD'] - df['Signal']
  plt.figure(figsize=(12, 6))
  plt.plot(df['MACD'], label='MACD', color='blue')
  plt.plot(df['Signal'], label='Signal Line', color='orange')
  plt.bar(df.index, df['Histogram'], label='Histogram', color='gray')
  plt.title('MACD Indicator')
  plt.xlabel('Date')
  plt.ylabel('Value')
  plt.show()
  df["Signals"] = 0
  df.loc[df['MACD'] > df['Signal'], 'MACD_Signal'] = 1
  df.loc[df['MACD'] < df['Signal'], 'MACD_Signal'] = -1
  return df["Signals"]


In [6]:
def stochastic_oscillator(df,long_period=14,short_period=3,column="Close"):
  df['Lowest_Low'] = df['Low'].rolling(window=long_period).min()
  df['Highest_High'] = df['High'].rolling(window=long_period).max()
  df['%K'] = 100 * ((df['Close'] - df['Lowest_Low']) / (df['Highest_High'] - df['Lowest_Low']))
  df['%D'] = df['%K'].rolling(window=short_period).mean()
  plt.figure(figsize=(12, 6))
  plt.plot(df['%K'], label='%K', color='blue')
  plt.plot(df['%D'], label='%D (Signal)', color='red')
  plt.axhline(80, color='gray', linestyle='--', label='Overbought (80)')
  plt.axhline(20, color='gray', linestyle='--', label='Oversold (20)')
  plt.title('Stochastic Oscillator')
  plt.xlabel('Date')
  plt.ylabel('Stochastic Value')
  plt.show()
  df["Signals"]=0
  df.loc[(df['%K'] < 20) & (df['%D'] < 20), 'Signals'] = 1
  df.loc[(df['%K'] > 80) & (df['%D'] > 80), 'Signals'] = -1
  return df['Signals']

In [7]:
def VWAP(df):
  df["Typical_Price"]=(df["Close"]+df["High"]+df["Low"])/3
  df["VWAP"]=(df["Typical_price"]*df["Volume"]).cumsum()/df["Volume"].cumsum()
  plt.figure(figsize=(12, 6))
  plt.plot(df['Close'], label='Close Price', color='blue')
  plt.plot(df['VWAP'], label='VWAP', color='orange')
  plt.title('VWAP Indicator')
  plt.xlabel('Timestamp')
  plt.ylabel('Price')
  plt.legend()
  plt.show()
  return df

In [8]:
def RSI(df,window=14):
  df["Returns"]=df["Close"].diff()
  gains=df["Returns"].where(df["Returns"]>0,0)
  loss=df["Returns"].where(df["Returns"]<0,0)
  rs=gains.rolling(window=window).mean()/loss.rolling(window=window).mean()
  df["RSI"]=100-100/(1+rs)
  plt.figure(figsize=(12, 6))
  plt.plot(df['RSI'], label='RSI', color='purple')
  plt.axhline(70, color='red', linestyle='--', label='Overbought (70)')
  plt.axhline(30, color='green', linestyle='--', label='Oversold (30)')
  plt.title('Relative Strength Index (RSI)')
  plt.xlabel('Date')
  plt.ylabel('RSI Value')
  plt.legend()
  plt.show()
  df["Signals"]=0
  df.loc[df["RSI"]>70,"Signals"]=-1
  df.loc[df["RSI"]<30,"Signals"]=1
  return df["Signals"]


In [9]:
def calculate_atr(df, period=14):
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
    df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].ewm(span=period, adjust=False).mean()
    plt.figure(figsize=(12, 6))
    plt.plot(df['ATR'], label=f'ATR ({period})', color='orange')
    plt.title('Average True Range (ATR)')
    plt.xlabel('Date')
    plt.ylabel('ATR')
    plt.grid(True)
    plt.legend()
    plt.show()
    return df



In [10]:
def clean(df):
    cleaned_signals = []
    current_pos = 0  # 0 = no position, 1 = long, -1 = short

    for i in range(len(df)):
        signal = df["Signals"].iloc[i]

        if current_pos == 0:
            cleaned_signals.append(signal)
            if signal != 0:
                current_pos = signal  # Enter position
        elif current_pos == 1:
            if signal == -1:  # Exit long
                cleaned_signals.append(signal)
                current_pos = 0
            else:
                cleaned_signals.append(0)  # Hold long
        elif current_pos == -1:
            if signal == 1:  # Exit short
                cleaned_signals.append(signal)
                current_pos = 0
            else:
                cleaned_signals.append(0)  # Hold short

    df["Cleaned_Signals"] = cleaned_signals
    return df

In [11]:
import pandas as pd
import matplotlib.pyplot as plt

def backtest(signals, df, initial_capital=1000, stop_pct=0.05):
    df = df.copy()
    df["Returns"] = df["Close"].pct_change().fillna(0)

    position = 0
    entry_price = 0
    cash = initial_capital
    equity_curve = []
    trade_log = []
    entry_index = None  # Track entry index for short position

    for i in range(len(df)):
        price = df["Close"].iloc[i]
        signal = signals[i]
        date = df.index[i]

        # ENTRY LOGIC
        if signal == 1 and position == 0:
            position = 1
            entry_price = price
            entry_index = i
            trade_log.append({"Entry_Date": date, "Entry_Price": price, "Position": "Long"})

        elif signal == -1 and position == 0:
            position = -1
            entry_price = price
            entry_index = i
            trade_log.append({"Entry_Date": date, "Entry_Price": price, "Position": "Short"})

        # EXIT LOGIC
        elif signal == -1 and position == 1:  # Exit long
            pnl = price - entry_price
            cash += pnl
            trade_log[-1].update({"Exit_Date": date, "Exit_Price": price, "PnL": pnl})
            position = 0
            entry_index = None

        elif signal == 1 and position == -1:  # Exit short
            pnl = entry_price - price
            cash += pnl
            trade_log[-1].update({"Exit_Date": date, "Exit_Price": price, "PnL": pnl})
            position = 0
            entry_index = None

        # Square off short position after 5 days
        elif position == -1 and entry_index is not None and (i - entry_index) >= 5:
            pnl = entry_price - price
            cash += pnl
            trade_log[-1].update({"Exit_Date": date, "Exit_Price": price, "PnL": pnl, "Forced_Exit": True})
            position = 0
            entry_index = None

        # Update equity
        if position == 1:
            equity = cash + (price - entry_price)
        elif position == -1:
            equity = cash + (entry_price - price)
        else:
            equity = cash

        equity_curve.append(equity)

    df['Equity'] = equity_curve
    df['Daily Return'] = df['Equity'].pct_change().fillna(0)

    # Performance metrics
    total_return = (df['Equity'].iloc[-1] - initial_capital) / initial_capital
    max_drawdown = (df['Equity'].cummax() - df['Equity']).max() / df['Equity'].cummax().max()
    sharpe_ratio = df['Daily Return'].mean() / df['Daily Return'].std() * (252**0.5)

    print(f"Total Return: {total_return * 100:.2f}%")
    print(f"Max Drawdown: {max_drawdown * 100:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['Equity'], label='Equity Curve')
    plt.title('Backtest Using Cleaned Signals')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.grid(True)
    plt.legend()
    plt.show()

    trades_df = pd.DataFrame(trade_log)
    return df, trades_df














